In [1]:
import os
import ast
import textwrap
import tiktoken
import matplotlib.pyplot as plt
import numpy as np
import umap.umap_ as umap
import pandas as pd
import shutil
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from dotenv import load_dotenv, find_dotenv
from typing import Optional

# ¿Cómo funciona O-Llama y Groq?

In [2]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings(
)

detailed_turbo_llm = turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-0125'
)

turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

# Eliminar la primera página o portada del doc.

In [3]:
# Carga de documentos y extracción de información
# (asegúrate de que haya PDFs en la carpeta documentos)
documents = DirectoryLoader('./docs/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')

docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

In [4]:
def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# counts = [num_tokens_from_string(t) for t in texts]

# plt.figure(figsize=(10, 6))
# plt.hist(counts, bins=30, color="blue", edgecolor="black", alpha=0.7)
# plt.title("Histograma: Cantidad de tokens")
# plt.xlabel("Cantidad de tokens")
# plt.ylabel("Frequencia")
# plt.show()

In [5]:
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Num tokens in all context: %s"
    % num_tokens_from_string(concatenated_content)
)

Num tokens in all context: 18836


# 1. ¿Incrustar de cero?
# 2. ¿Elegir un encrustamiento guardado?

In [6]:
user_input = input("Elige 1 o 2: ")

if user_input.lower() == "1":
    print('Elegiste 1')
    global_embeddings = [embeddings.embed_query(txt) for txt in texts]

    embed_name = input('¿Cómo se llama esta incrustación?: ') + '_emb' + '.txt'
    with open(rf'./{embed_name}', 'w') as f:
        for i in global_embeddings:
            f.write("%s\n" % i)
    
elif user_input.lower() == "2":
    print('Elegiste 2')
    global_embeddings = []

    embed_name = input('Nombre del encrustamiento: ') + '_emb' + '.txt'

    with open(rf'./embed/{embed_name}', 'r') as f:
        for i in f:
            x = ast.literal_eval(i.strip())  # Convertir la cadena a lista de números
            global_embeddings.append(x)

    global_embeddings = np.array(global_embeddings, dtype=float)
    
elif user_input != "1" and user_input != "2":
    print("No seleccionaste ningún encrustamiento.")

Elegiste 1


In [7]:
def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

dim = 2
global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)

# plt.figure(figsize=(10, 8))
# plt.scatter(global_embeddings_reduced[:, 0], global_embeddings_reduced[:, 1], alpha=0.5)
# plt.title("Global Embeddings")
# plt.xlabel("Dimension 1")
# plt.ylabel("Dimension 2")
# plt.show()

In [8]:
def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters

labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)

# plot_labels = np.array([label[0] if len(label) > 0 else -1 for label in labels])
# plt.figure(figsize=(10, 8))

# unique_labels = np.unique(plot_labels)
# colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

# for label, color in zip(unique_labels, colors):
#     mask = plot_labels == label
#     plt.scatter(global_embeddings_reduced[mask, 0], global_embeddings_reduced[mask, 1], color=color, label=f'Cluster {label}', alpha=0.5)

# plt.title("Cluster Visualization of Global Embeddings")
# plt.xlabel("Dimension 1")
# plt.ylabel("Dimension 2")
# plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
# plt.tight_layout(rect=[0, 0, 0.75, 1])

c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [9]:
simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

df = pd.DataFrame({
    'Texto': texts,
    'Embedding': list(global_embeddings_reduced),
    'Cluster': simple_labels
})
# print(df.head(3))
df.head()

,Texto,Embedding,Cluster
0,La metodología Lean Startup : desarrollo y a...,"[2.2112026, 6.584024]",26
1,Un aprendizaje empírico que parte de unas hipó...,"[3.1075935, 4.2927074]",3
2,Francisco Javier Llamas Fernández / Juan Carlo...,"[1.6168774, 6.313115]",10
3,"expérimentation, devant permettre aux entrepre...","[1.4404036, 5.9832916]",30
4,81 Rev. esc.adm.neg. La metodología Lean start...,"[1.898208, 6.861736]",11


In [10]:
def format_cluster_texts(df):
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Texto'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts

clustered_texts = format_cluster_texts(df)
clustered_texts

{26: 'La metodología Lean  Startup :  desarrollo y aplicación para el emprendimiento 1 Doctorando en Ciencias Sociales, Universidad Pontificia de Salamanca; Magister en Marketing Digital y Publicidad,  Universidad Nebrija. Fundador y director académico, European School of Leadership (EUschool) 2 Doctor en Psicología por la Universidad Complutense de Madrid. En la actualidad es director del grado de psicología  de la Universidad Antonio de Nebrija. Ha sido profesor en varias universidades españolas y profesor invitado en varias  universidades latinoamericanas. ORCID: https://orcid.org/0000-0003- 3312-861XDOI: https://doi.org/10.21158/01208160.n84.2018.1918Francisco Javier Llamas Fernández1 European School of Leadership franciscojllamas@hotmail.com Juan Carlos Fernández Rodríguez2  Universidad Antonio de Nebrija jfernanr@nebrija.es Cómo citar este artículo / To reference this article / Comment citer cet article / Para citar este artigo:   Llamas Fernández, F . J. y Fernández Rodríguez, J

In [11]:
template = """Eres un revisor bibliográfico profesional encargado de crear resúmenes extremadamente detallados del texto proporcionado.

Texto:
{text}
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | detailed_turbo_llm | StrOutputParser()

In [12]:
summaries = {}
for cluster, text in clustered_texts.items():
    summary = chain.invoke({"text": text})
    summaries[cluster] = summary

summaries

{26: ' mínimo viable (MVP) que se lanza al mercado para obtener feedback de los usuarios. Este feedback permite pivotar o perseverar en la idea inicial, adaptándola a las necesidades reales del mercado.\n\nLa metodología Lean Startup se basa en la construcción de un ciclo de aprendizaje continuo, donde se van realizando iteraciones rápidas y se van ajustando las hipótesis iniciales en función de los resultados obtenidos. Se destaca la importancia de la experimentación, la medición y el aprendizaje constante como pilares fundamentales para el éxito de un emprendimiento.\n\nAdemás, se aborda la importancia de la innovación disruptiva, la flexibilidad y la adaptabilidad como características clave para sobrevivir en un entorno empresarial cada vez más competitivo y cambiante. Se mencionan también algunas herramientas y técnicas que pueden ser útiles para implementar la metodología Lean Startup de manera efectiva.\n\nEn resumen, la metodología Lean Startup se presenta como una forma innovad

In [13]:
embedded_summaries = [embeddings.embed_query(summary) for summary in summaries.values()]

embedded_summaries_np = np.array(embedded_summaries)

labels, _ = gmm_clustering(embedded_summaries_np, threshold=0.5)

simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

In [14]:
clustered_summaries = {}
for i, label in enumerate(simple_labels):
    if label not in clustered_summaries:
        clustered_summaries[label] = []
    clustered_summaries[label].append(list(summaries.values())[i])

clustered_summaries

{0: [' mínimo viable (MVP) que se lanza al mercado para obtener feedback de los usuarios. Este feedback permite pivotar o perseverar en la idea inicial, adaptándola a las necesidades reales del mercado.\n\nLa metodología Lean Startup se basa en la construcción de un ciclo de aprendizaje continuo, donde se van realizando iteraciones rápidas y se van ajustando las hipótesis iniciales en función de los resultados obtenidos. Se destaca la importancia de la experimentación, la medición y el aprendizaje constante como pilares fundamentales para el éxito de un emprendimiento.\n\nAdemás, se aborda la importancia de la innovación disruptiva, la flexibilidad y la adaptabilidad como características clave para sobrevivir en un entorno empresarial cada vez más competitivo y cambiante. Se mencionan también algunas herramientas y técnicas que pueden ser útiles para implementar la metodología Lean Startup de manera efectiva.\n\nEn resumen, la metodología Lean Startup se presenta como una forma innovad

In [15]:
final_summaries = {}
for cluster, texts in clustered_summaries.items():
    combined_text = ' '.join(texts)
    summary = chain.invoke({"text": combined_text})
    final_summaries[cluster] = summary

final_summaries

{0: 'El texto aborda la metodología Lean Startup, que se centra en la creación de un Producto Mínimo Viable (PMV) como paso inicial en el proceso de emprendimiento. Se destaca la importancia de validar hipótesis a través de la experimentación y la interacción con los clientes desde las etapas iniciales del desarrollo del producto. Se menciona la necesidad de pivotar o perseverar en función de la retroalimentación recibida, adaptando el producto a las necesidades reales del mercado.\n\nSe resalta la importancia de la experimentación, la medición y el aprendizaje constante como pilares fundamentales para el éxito de un emprendimiento. Se menciona la importancia de la innovación disruptiva, la flexibilidad y la adaptabilidad como características clave para sobrevivir en un entorno empresarial competitivo y cambiante.\n\nAdemás, se abordan herramientas y técnicas útiles para implementar la metodología Lean Startup de manera efectiva. Se destaca la importancia de identificar las característ

In [16]:
texts_from_df = df['Texto'].tolist()
texts_from_clustered_texts = list(clustered_texts.values())
texts_from_final_summaries = list(final_summaries.values())

combined_texts = texts_from_df + texts_from_clustered_texts + texts_from_final_summaries

In [ ]:
combined_texts[:10]

In [17]:
# Nombre del archivo de texto
file_name = input('Nombre del archivo: ') + '.txt'

# Escribir la lista en el archivo de texto
with open(file_name, 'w', encoding='utf-8') as f:
    for t in combined_texts:
        f.write("%s\n" % t)

# Leer el contenido del archivo y mostrarlo
with open(file_name, 'r', encoding='utf-8') as f:
    content = f.read()

textos = text_splitter.split_text(content)

# Guardar los kb en una carpeta (kbs)

### 1. Enseñar

In [18]:
import time
destino_kb = r'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\kbs'

In [28]:
persist_directory = input('¿Cómo se llamará este knowledge-base?: ') + '_kb'
vectorstore = Chroma.from_texts(texts=textos,
                                embedding=embeddings,
                                persist_directory=persist_directory)
vectorstore.persist()
vectorstore = None
os.system(f'zip -r db.zip ./{persist_directory}')
time.sleep(1)

try:
    shutil.move(persist_directory, destino_kb)
except PermissionError as e:
    pass

### 2. Recordar (si y solo si Enseñar=Done) y pregunta de control

In [ ]:
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
    final_number = initial_number
    while final_number < max_threshold:
        retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
        docs = retriever.invoke(string)
        text = "".join([doc.page_content for doc in docs])
        if num_tokens_from_string(text) < max_threshold:
            final_number += 1
        else:
            break
    return final_number

final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
print(final_number)
retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})

In [ ]:
# Se personaliza el LLM
template = """
Eres Prometeo, un asistente personal de revisión biliográfica que habla Español.

Tu tarea consiste en:

1. Leer detalladamente la información proporcionada en documentos que generalmente son
artículos científicos en formato PDF.

2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
con el contexto, especialmente a preguntas que involucren peticiones. Evita concluir tus respuestas con un 
párrafo que comience con 'En resumen,...'. Varía las conclusiones de tus respuestas para que 
sean más diversas y creativas.

3. Ser carismático y ofrecer información sobre ti y tus funciones.

SIEMPRE debes responder en Español.

SIEMPRE debes responder con base al siguiente contexto: {context}.

NUNCA hables específicamente del contexto.

NUNCA terminar las respuestas con un resumen o una conclusión predecible.

Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prometeo_prompt
    | turbo_llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain
)

In [ ]:
def wrap_text_preserve_newlines(text, width=70):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['answer']))
    print('\nReferencias:')
    for contexto in llm_response["context"][:5]:
        print(contexto)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

# Hacer historial y guardar conversaciones para medir efectividad.

In [ ]:
# Para archivos grandes debes dividir el texto

# # Diccionario inicial
# data = {
#     2: 'El impacto de la innovación de producto, innovación de proceso, innovación organizacional y la percepción de la corrupción en la rentabilidad empresarial en Colombia es un tema abordado en un estudio realizado por Gueiriluz Romero Chaparro y Juan Felipe Gómez Quiceno de la Universidad EAN. El propósito de la investigación es analizar'
# }

# # Obtener el texto original
# original_text = data[2]

# # Encontrar el punto medio
# mid_point = len(original_text) // 2

# # Definir el tamaño del solapamiento
# overlap_size = 50

# # Calcular los puntos de división con solapamiento
# start_overlap = max(mid_point - overlap_size // 2, 0)
# end_overlap = min(mid_point + overlap_size // 2, len(original_text))

# # Dividir el texto en dos partes con solapamiento
# first_half = original_text[:end_overlap]
# second_half = original_text[start_overlap:]

# # Crear nuevas llaves en el diccionario
# data[2] = first_half
# data[3] = second_half

# # Imprimir el diccionario actualizado
# print(data)